# 📓 Notebook 3 : Evaluate & Predict
CropHealth Detection - Évaluation comparative et prédictions

📌 Cellule 1 : Vérifier checkpoints

In [ ]:
import glob
import torch
from pathlib import Path
import pandas as pd

print("="*70)
print("🔍 VÉRIFICATION DES CHECKPOINTS")
print("="*70)

# Check GPU
assert torch.cuda.is_available(), "❌ GPU non disponible !"
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

# Chercher checkpoints
checkpoints = {
    'ssd': sorted(glob.glob('runs/CropHealth_SSD_*/best.pt')),
    'yolov8n': sorted(glob.glob('runs/CropHealth_YOLOv8n_*/weights/best.pt')),
    'efficientdet': sorted(glob.glob('runs/CropHealth_EfficientDet_*/best.pt')),
    'fasterrcnn': sorted(glob.glob('runs/CropHealth_FasterRCNN_*/best.pt')),
    'fasterrcnn_light': sorted(glob.glob('runs/CropHealth_FasterRCNN_light_*/best.pt')),
}

# Tableau résumé
checkpoints_summary = []
for model, ckpt_list in checkpoints.items():
    if ckpt_list:
        latest = ckpt_list[-1]
        checkpoints_summary.append({
            'Modèle': model,
            'Checkpoint': latest,
            'Status': '✅'
        })
    else:
        checkpoints_summary.append({
            'Modèle': model,
            'Checkpoint': 'N/A',
            'Status': '❌'
        })

df = pd.DataFrame(checkpoints_summary)
print(f"\n{df.to_string(index=False)}")

# Compter disponibles
available = sum(1 for item in checkpoints_summary if item['Status'] == '✅')
print(f"\n✅ {available}/5 modèles disponibles pour évaluation")
print("="*70)

📌 Cellule 2 : Évaluation multi-modèles

In [ ]:
print("\n" + "="*70)
print("📊 ÉVALUATION COMPARATIVE DES MODÈLES")
print("="*70 + "\n")

# Construire commande avec checkpoints disponibles
checkpoint_args = []
for model, ckpt_list in checkpoints.items():
    if ckpt_list:
        checkpoint_args.append(f"{model}:{ckpt_list[-1]}")

if len(checkpoint_args) > 0:
    checkpoint_str = ' '.join(checkpoint_args)

    print(f"🔄 Évaluation de {len(checkpoint_args)} modèles...")
    print("⏳ Cette étape peut prendre 10-15 minutes\n")

    !python evaluate_models.py \
        --checkpoints {checkpoint_str} \
        --val-data data/yolo_crop \
        --output evaluation_results \
        --device cuda

    print("\n✅ Évaluation terminée !")
else:
    print("❌ Aucun checkpoint disponible !")
    print("💡 Retourner au Notebook 2 pour entraîner les modèles")

📌 Cellule 3 : Afficher métriques globales

# 📌 Cellule 3 : Afficher métriques globales

In [ ]:
from IPython.display import display
import pandas as pd

print("\n" + "="*70)
print("📈 MÉTRIQUES GLOBALES")
print("="*70 + "\n")

# Lire CSV métriques globales
csv_path = 'evaluation_results/global_metrics.csv'

if Path(csv_path).exists():
    df_metrics = pd.read_csv(csv_path)

    # Styler le DataFrame
    styled_df = df_metrics.style.background_gradient(
        subset=['mAP@50 (%)', 'F1-Score (%) - Micro'],
        cmap='Greens'
    ).format({
        'mAP@50 (%)': '{:.2f}',
        'Precision (%) - Micro': '{:.2f}',
        'Recall (%) - Micro': '{:.2f}',
        'F1-Score (%) - Micro': '{:.2f}'
    })

    display(styled_df)

    # Trouver meilleur modèle
    best_model_idx = df_metrics['mAP@50 (%)'].idxmax()
    best_model = df_metrics.iloc[best_model_idx]

    print(f"\n🏆 MEILLEUR MODÈLE: {best_model['Model']}")
    print(f"   • mAP@50: {best_model['mAP@50 (%)']:.2f}%")
    print(f"   • F1-Score: {best_model['F1-Score (%) - Micro']:.2f}%")
else:
    print("❌ Fichier global_metrics.csv non trouvé")
    print("💡 Vérifier que l'évaluation s'est bien terminée")

print("="*70)

📌 Cellule 4 : Afficher AP@50 par classe

In [ ]:
print("\n" + "="*70)
print("📊 AP@50 PAR CLASSE")
print("="*70 + "\n")

csv_path = 'evaluation_results/ap50_per_class.csv'

if Path(csv_path).exists():
    df_ap = pd.read_csv(csv_path)

    # Convertir en numérique
    for col in df_ap.columns[1:]:
        df_ap[col] = pd.to_numeric(df_ap[col], errors='coerce')

    # Styler
    styled_ap = df_ap.style.background_gradient(
        subset=df_ap.columns[1:],
        cmap='YlOrRd'
    ).format({col: '{:.2f}' for col in df_ap.columns[1:]})

    display(styled_ap)

    # Classes difficiles
    mean_ap = df_ap[df_ap.columns[1:]].mean(axis=1)
    df_ap['Moyenne'] = mean_ap

    difficult_classes = df_ap.nsmallest(3, 'Moyenne')['Class'].tolist()
    print(f"\n⚠️  Classes les plus difficiles: {', '.join(difficult_classes)}")
else:
    print("❌ Fichier ap50_per_class.csv non trouvé")

print("="*70)

📌 Cellule 5 : Afficher F1-Score par classe

In [ ]:
print("\n" + "="*70)
print("📊 F1-SCORE PAR CLASSE")
print("="*70 + "\n")

csv_path = 'evaluation_results/f1_per_class.csv'

if Path(csv_path).exists():
    df_f1 = pd.read_csv(csv_path)

    # Filtrer lignes F1-Score uniquement
    df_f1_only = df_f1[df_f1['Metric'] == 'F1-Score (%)'].copy()
    df_f1_only = df_f1_only.drop('Metric', axis=1)

    # Convertir en numérique
    for col in df_f1_only.columns[1:]:
        df_f1_only[col] = pd.to_numeric(df_f1_only[col], errors='coerce')

    # Styler
    styled_f1 = df_f1_only.style.background_gradient(
        subset=df_f1_only.columns[1:],
        cmap='Blues'
    ).format({col: '{:.2f}' for col in df_f1_only.columns[1:]})

    display(styled_f1)
else:
    print("❌ Fichier f1_per_class.csv non trouvé")

print("="*70)

📌 Cellule 6 : Visualiser graphiques comparatifs

In [ ]:
from IPython.display import Image as IPImage, display

print("\n" + "="*70)
print("📈 GRAPHIQUES COMPARATIFS")
print("="*70 + "\n")

plots = {
    'mAP@50 Global': 'evaluation_results/map50_comparison.png',
    'AP@50 par Classe': 'evaluation_results/ap50_per_class_comparison.png',
    'F1-Score par Classe': 'evaluation_results/f1_per_class_comparison.png',
}

for title, path in plots.items():
    if Path(path).exists():
        print(f"📊 {title}:")
        display(IPImage(filename=path, width=800))
        print("\n")
    else:
        print(f"❌ {title}: Non trouvé\n")

📌 Cellule 7 : Visualiser Precision-Recall curves

In [ ]:
print("\n" + "="*70)
print("📈 PRECISION-RECALL CURVES (3 premières classes)")
print("="*70 + "\n")

pr_curves = sorted(glob.glob('evaluation_results/pr_curve_*.png'))[:3]

for pr_path in pr_curves:
    class_name = Path(pr_path).stem.replace('pr_curve_', '')
    print(f"📊 Classe: {class_name}")
    display(IPImage(filename=pr_path, width=700))
    print("\n")

if len(pr_curves) > 3:
    print(f"💡 {len(pr_curves) - 3} autres courbes PR disponibles dans evaluation_results/")

📌 Cellule 8 : Visualiser Confusion Matrices

In [ ]:
print("\n" + "="*70)
print("📊 CONFUSION MATRICES")
print("="*70 + "\n")

cm_files = sorted(glob.glob('evaluation_results/confusion_matrix_*.png'))

for cm_path in cm_files:
    model_name = Path(cm_path).stem.replace('confusion_matrix_', '')
    print(f"📊 {model_name.upper()}:")
    display(IPImage(filename=cm_path, width=700))
    print("\n")

📌 Cellule 9 : Prédictions sur images de test

In [ ]:
print("\n" + "="*70)
print("🔍 PRÉDICTIONS SUR IMAGES DE TEST")
print("="*70 + "\n")

# Utiliser meilleur modèle
if 'best_model' in locals():
    model_name = best_model['Model'].lower()
    checkpoint = checkpoints[model_name][-1] if checkpoints[model_name] else None

    if checkpoint:
        print(f"📦 Utilisation du meilleur modèle: {model_name}")
        print(f"📁 Checkpoint: {checkpoint}\n")

        # Créer dossier test si n'existe pas
        test_dir = 'data/yolo_crop/val/images'

        if model_name == 'yolov8n':
            # YOLOv8n
            !python yolo_predict.py \
                --checkpoint {checkpoint} \
                --input {test_dir} \
                --data-yaml data/yolo_crop/data.yaml \
                --output predictions/{model_name} \
                --conf 0.5
        else:
            # Autres modèles
            !python predict.py \
                --model {model_name} \
                --checkpoint {checkpoint} \
                --input {test_dir} \
                --val-data data/yolo_crop \
                --output predictions/{model_name} \
                --conf 0.5

        print(f"\n✅ Prédictions sauvegardées dans: predictions/{model_name}/")
    else:
        print(f"❌ Checkpoint non trouvé pour {model_name}")
else:
    print("❌ Meilleur modèle non défini")

📌 Cellule 10 : Visualiser prédictions

In [ ]:
from IPython.display import Image as IPImage, display
import random

print("\n" + "="*70)
print("🖼️  EXEMPLES DE PRÉDICTIONS")
print("="*70 + "\n")

if 'model_name' in locals():
    pred_dir = Path(f'predictions/{model_name}')

    if pred_dir.exists():
        # Lister images prédites
        pred_images = list(pred_dir.glob('*.jpg')) + list(pred_dir.glob('*.png'))

        # Afficher 5 images aléatoires
        if pred_images:
            random.shuffle(pred_images)

            for img_path in pred_images[:5]:
                print(f"📷 {img_path.name}:")
                display(IPImage(filename=str(img_path), width=700))
                print("\n")

            if len(pred_images) > 5:
                print(f"💡 {len(pred_images) - 5} autres prédictions disponibles dans {pred_dir}")
        else:
            print("❌ Aucune prédiction trouvée")
    else:
        print(f"❌ Dossier {pred_dir} non trouvé")

📌 Cellule 11 : Export modèles (ONNX)

In [ ]:
print("\n" + "="*70)
print("📦 EXPORT MODÈLES (ONNX)")
print("="*70 + "\n")

if 'model_name' in locals() and model_name != 'yolov8n':
    print(f"🔄 Export {model_name} vers ONNX...")

    !python export/export_models.py \
        --model {model_name} \
        --checkpoint {checkpoint} \
        --format onnx \
        --output exports/{model_name}

    print(f"\n✅ ONNX export terminé: exports/{model_name}/")
elif model_name == 'yolov8n':
    print(f"🔄 Export YOLOv8n vers ONNX...")

    !python export/export_models.py \
        --model yolov8n \
        --checkpoint {checkpoint} \
        --format onnx \
        --output exports/yolov8n

    print(f"\n✅ ONNX export terminé: exports/yolov8n/")
else:
    print("⚠️  Aucun modèle à exporter")

📌 Cellule 12 : Télécharger résultats

In [ ]:
import shutil
from google.colab import files

print("\n" + "="*70)
print("📥 PRÉPARATION DU TÉLÉCHARGEMENT")
print("="*70 + "\n")

# Créer archives
archives = []

if Path('evaluation_results').exists():
    print("📦 Compression evaluation_results...")
    shutil.make_archive('crophealth_evaluation', 'zip', 'evaluation_results')
    archives.append('crophealth_evaluation.zip')

if Path('predictions').exists():
    print("📦 Compression predictions...")
    shutil.make_archive('crophealth_predictions', 'zip', 'predictions')
    archives.append('crophealth_predictions.zip')

if Path('exports').exists():
    print("📦 Compression exports...")
    shutil.make_archive('crophealth_exports', 'zip', 'exports')
    archives.append('crophealth_exports.zip')

print(f"\n✅ {len(archives)} archives créées")

# Télécharger
print("\n📥 Téléchargement des archives...")
for archive in archives:
    files.download(archive)
    print(f"  ✅ {archive}")

print("\n✅ Téléchargement terminé !")

📌 Cellule 13 : Sauvegarder vers Google Drive

In [ ]:
from datetime import datetime

print("\n" + "="*70)
print("💾 SAUVEGARDE VERS GOOGLE DRIVE")
print("="*70 + "\n")

timestamp = datetime.now().strftime('%Y%m%d_%H%M')
backup_dir = f'/content/drive/MyDrive/CropHealth_Results_{timestamp}'

print(f"📁 Création du backup: {backup_dir}")
Path(backup_dir).mkdir(parents=True, exist_ok=True)

# Copier résultats
for folder in ['evaluation_results', 'predictions', 'exports']:
    if Path(folder).exists():
        print(f"📦 Copie {folder}...")
        shutil.copytree(folder, f'{backup_dir}/{folder}', dirs_exist_ok=True)

print(f"\n✅ Backup terminé !")
print(f"📁 Location: {backup_dir}")

📌 Cellule 14 : Résumé final du projet

In [ ]:
import json

print("\n" + "="*70)
print("🎉 PROJET CROPHEALTH DETECTION - RÉSUMÉ FINAL")
print("="*70 + "\n")

final_summary = {
    "Modèles entraînés": available,
    "Meilleur modèle": {
        "Nom": best_model['Model'] if 'best_model' in locals() else 'N/A',
        "mAP@50": f"{best_model['mAP@50 (%)']:.2f}%" if 'best_model' in locals() else 'N/A',
        "F1-Score": f"{best_model['F1-Score (%) - Micro']:.2f}%" if 'best_model' in locals() else 'N/A'
    },
    "Évaluation": {
        "Métriques calculées": "mAP@50, AP par classe, F1-Score, PR curves, Confusion matrices",
        "Résultats": "evaluation_results/"
    },
    "Prédictions": {
        "Images testées": len(list(Path(f'predictions/{model_name}').glob('*.jpg'))) if 'model_name' in locals() and Path(f'predictions/{model_name}').exists() else 0,
        "Output": f"predictions/{model_name}/" if 'model_name' in locals() else 'N/A'
    },
    "Export": {
        "Format": "ONNX",
        "Location": f"exports/{model_name}/" if 'model_name' in locals() else 'N/A'
    },
    "Backup Drive": backup_dir
}

print(json.dumps(final_summary, indent=2, ensure_ascii=False))

print("\n" + "="*70)
print("✅ PROJET TERMINÉ AVEC SUCCÈS !")
print("="*70)
print("\n📊 Résultats disponibles:")
print("   • Tableaux CSV: evaluation_results/")
print("   • Graphiques: evaluation_results/*.png")
print("   • Prédictions: predictions/")
print("   • Exports ONNX: exports/")
print("   • Backup Drive: voir ci-dessus")
print("\n🎓 Prêt pour présentation PFE !")